Custom Tool

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.agents import AgentType

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

In [6]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "WMI search"
    description = "useful when you need to answer questions about WMi product"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [7]:
from langchain.agents import initialize_agent
from langchain_openai import OpenAI

llm=OpenAI()

tools = [CustomSearchTool()]

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

/Users/sumgupta/Data/Rag_pixegami/langchain-rag-tutorial/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [40]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

WMI search - useful when you need to answer questions about WMi product

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [WMI search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [41]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=10)

In [42]:
agent.run("Who is Sachin Tendulkar?")



> Entering new AgentExecutor chain...
 Sachin Tendulkar is a well-known cricketer, so I should use WMI search to find more information about him.
Action: WMI search
Action Input: Sachin Tendulkar

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID cc771c2a-9047-405e-8a99-8759043a86b1, but expected {'tool'} run.")



Observation: ...............................
......................................................
.............................................
..............................................
Thought: There are many articles about Sachin Tendulkar, I should refine my search to find more specific information.
Action: WMI search
Action Input: Sachin Tendulkar cricket

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 3111aa1d-d104-4610-a558-6722afd2d532, but expected {'tool'} run.")



Observation: ...............................
.............................................
..............................................
.....................................................
Thought: I now know that Sachin Tendulkar is a retired Indian international cricketer who is widely regarded as one of the greatest batsmen in the history of cricket.
Final Answer: Sachin Tendulkar is a retired Indian international cricketer regarded as one of the greatest batsmen in the history of cricket.

> Finished chain.


'Sachin Tendulkar is a retired Indian international cricketer regarded as one of the greatest batsmen in the history of cricket.'

In [31]:
agent.run("Does WMi support json interfaces?")



> Entering new AgentExecutor chain...
 You should always think about what to do
Action:
WMI search
Action Input: json

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID d7baff70-6cd5-4ff3-8937-f607fc3405db, but expected {'tool'} run.")



Observation: JSON Support for Interfaces
Appointments ............................................................................... 3 JSON
................................................................
................................
Thought: WMI can support json interfaces
Final Answer: Yes, WMI does support json interfaces.

> Finished chain.


'Yes, WMI does support json interfaces.'